## Priprema podataka

In [2]:
import pandas as pd
import numpy as np
import collections
import time

In [3]:
import xgboost as xgb
import lightgbm as lgb
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from imblearn.ensemble import EasyEnsembleClassifier

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline

In [5]:
data = pd.read_csv('creditcard.csv')

In [6]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


Da bismo mogli učiti na podacima moramo ih pripremiti tako da budu u pogodnom obliku. Prije svega, pošto znamo da je nad anonimiziranim značajkama provedena PCA transformacija, moramo skalirati i značajku 'Amount'. <br>
 Nakon skaliranja odbacujemo značajke 'Time' i 'Amount'. Umjesto toga u skup podataka vraćamo značajku 'Amount_Afte_Scaling'. <br>
 <b>Napomena</b>: Značajku 'Time' ćemo samo izbaciti iz skupa jer smo ranije zaključili da generalno nema veze sa tim je li neka transakcija valjana ili prevara. 

In [7]:
from sklearn.preprocessing import StandardScaler

amount_after_scaling = StandardScaler().fit_transform(data['Amount'].values.reshape(-1,1))

data.drop(['Time', 'Amount'], axis=1, inplace=True)
data.insert(0, 'Amount_after_scaling', amount_after_scaling)

Sljedeći korak koji trebamo je podijeliti podatke u skup za treniranje te u skup za testiranje. To nam omogućuje metoda iz sklearn biblioteke zvana train_test_split. Podjelu radimo tako da 20% podataka stavimo u skup za testiranje, dok za treniranje koristimo ostalih 80% podataka. 

In [8]:
from sklearn.model_selection import train_test_split

X = data.drop('Class', axis=1)
y = data['Class']
X_data_train, X_data_test, y_data_train, y_data_test = train_test_split(X,y,test_size=0.20,random_state=42)
y_data_test_non_fraud_num = y_data_test.value_counts()[0]
y_data_test_fraud_num = y_data_test.value_counts()[1]

Napomenimo još da ćemo za <i>cross-validation</i> koristiti <i>RepeatedStratifiedKFold</i> prilikom kojeg podatke dijelimo 5 puta te taj postupak ponavljamo 3 puta. 

In [9]:
from sklearn.model_selection import RepeatedStratifiedKFold
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)

### Parametri

In [ ]:
#popis klasifikatora koje koristimo

#### Logistička regresija

Parametri koje podešavamo kod linearne regresije su:
<ul>
    <li><em>C</em> - regularizacijski parametar, inverzno proporcionalan $\lambda$ regulatoru. </li>
    <li><em>penalty</em> - određuje koju regularizacijsku tehniku koristimo
        <ul>
            <li><i>ridge (l2)</i> - kazna je "kvadratna veličina" koeficjenata
                \[
                rCost = \sum_{i = 1}^{n}\bigg(y_i - \sum_{j=1}^{p}x_{ij}\beta_j \bigg)^2  + \lambda \sum_{j=1}^{p} \beta_j^2
                \]</li>
            <li><i>lasso (l1)</i> - kazna je "apsolutna vrijednost veličine" koeficjenata 
            \[
            lCost = \sum_{i = 1}^{n}\bigg(y_i - \sum_{j=1}^{p}x_{ij}\beta_j \bigg)^2  + \lambda \sum_{j=1}^{p} \mid\beta_j\mid
            \]</li>
        </ul>
    </li>
</ul>

In [12]:
from sklearn.model_selection import GridSearchCV
log_reg_c_params = [0.01, 0.1, 1, 10, 100]
log_reg_penalties = ['l1','l2']
log_reg_params = {"penalty": log_reg_penalties, 'C': log_reg_c_params}

#### Stabla odluke

Parametri koje podešavamo kod stabala odluke:
<ul>
    <li><em>kriterij</em> - funkcija kvalitete podjele
        <ul>
            <li><i>gini</i> - kriterij koji koristimo da smanjimo vjerojatnost pogrešne klasifikacije
            \[
            Gini = 1 - \sum_j p_j^2
            \]</li>
            <li><i>entropy</i> - kriterij koji koristimo da smanjimo nečistoću podjele
            \[
            Entropy = - \sum_j p_j log_2p_j
            \]</li>
        </ul>
    </li>
</ul>

In [13]:
criterion_params = ['gini', 'entropy']
splitter_params = ['best', 'random']
max_depth_params = [1, 2, 3, 4, 5, 6, 7]
min_samples_leaf_params = [1, 3, 5, 7]
dt_params = {'criterion': criterion_params, 'splitter': splitter_params, 'max_depth': max_depth_params, 'min_samples_leaf': min_samples_leaf_params}

#### Bagging klasifikator

Parametri koje podešavamo na bagging klasifikatoru na stablima odluke:
<ul>
    <li><em>broj procjenitelja</em> - određuje broj stabala odluke u ansamblu </li>
    <li><em>maksimalan udio podataka</em> - određuje udio koji 'vadimo' iz trening skupa kod treniranja svakog procjenitelja</li>
    <li><em>maksimalan udio značajki</em> - određuje udio značajki koje koristimo prilikom treniranja </li>
</ul>

In [14]:
num_estimators_params = [5, 7, 10, 12, 20, 50, 100]
max_samples_params = [0.5, 0.7, 0.9, 1.0]
max_features_params = [0.3, 0.5, 0.7, 0.9, 1.0]
bag_params = {'n_estimators': num_estimators_params, 'max_samples': max_samples_params,
              'max_features': max_features_params}

#### Slučajne šume

Parametri koje promatramo kod slučajnih šuma:
<ul>
    <li><em>kriterij</em> - poprima vrijednosti <i>gini</i> ili <i>entropy</i> koji su isto definirani kao i kod stabala odluke</li>
    <li><em>broj procjenitelja</em> - određuje broj stabala u slučajnoj šumi</li>
    <li><em>maksimalna dubina</em> - određuje maksimalnu dubinu svakog stabla u šumi</li>
    <li><em>minimalan broj list</em> - određuje koliko primjeraka najmanje smije bit u listu stabla</li>
    <li><em>težina klasa</em> - određuje težine koje dodjeljujemo klasama prilikom klasifikacije (valjana, prevara)
        <ul>
            <li><i>balanced</i> - određuje težine tako da uzima broj pojavljivanja klasa u ulaznim podacima</li>
            <li><i>balanced_subsample</i> - isto kao i <i>balanced</i>, ali su u ovom slučaju težine bazirane na bootstrap primjerku svakog stabla koje se gradi</li>
        </ul>
    </li>
</ul>

In [15]:
criterion_params = ['gini', 'entropy']
num_estimators_params = [5, 10, 50, 100]
max_depth_params = [1, 2, 3, 4, 5, 7, 9]
min_samples_leaf_params = [1, 3, 5, 7]
class_weight_params = ['balanced', 'balanced_subsample']
rf_params = {'criterion': criterion_params, 'n_estimators': num_estimators_params, 'max_depth': max_depth_params,
             'min_samples_leaf': min_samples_leaf_params, 'class_weight': class_weight_params}

#### Metoda najbližih susjeda

Parametri koje podešavamo kod metode najbližih susjeda:
<ul>
    <li><em>broj susjeda</em> - određuje koliko najbližih susjeda sudjeluje u 'glasanju' prilikom klasifikacije nekog primjerka</li>
    <li><em>algoritam</em> - određuje algoritam koji koristimo da bi pronašli najbliže susjede 
        <ul>
            <li><i>ball tree</i> - dijeli primjerke podataka u ugniježđene skupove hipersfera koje nazivamo loptama (balls)</li>
            <li><i>KD tree</i> - dijeli primjerke podataka u k-dimenzionalne kocke</li>
            <li><i>brute</i> - koristi dobro poznati pristup grubom silom</li>
            <li><i>auto</i> - automatski pokušava utvrditi koji je najprikladniji algoritam na temelju podataka koje prima <i>fit</i> metoda
        </ul>
    </li>
    <li><em>p (power)</em> - biramo koji algoritam koristimo da bi mjerili udaljenost
        <ul>
            <li><i>manhattan udaljenost (1)</i> - $d_1(p,q) = \sum_{i=1}^n \mid p_i - q_i \mid$</li>
            <li><i>euklidska udaljenost (2)</i> - $d_2(p,q) = \sqrt{\sum_{i=1}^n  (p_i - q_i)^2 }$</li>
        </ul>
    </li>
</ul>

In [16]:
num_neighbors_params = [1, 2, 3, 4, 5, 7, 9]
alg_params = ['auto', 'brute', 'ball_tree', 'kd_tree']
p_params = [1, 2]

knn_params = {'n_neighbors': num_neighbors_params, 'algorithm': alg_params, 'p': p_params, 'metric': ['minkowski']}

#### Metoda potpornih vektora

Parametri koje podešavamo za metodu potpornih vektora:
<ul>
    <li><em>C</em> - regularizacijski parametar, ima istu ulogu kao i u logističkoj regresiji</li>
    <li><em>kernel</em> - određuje funkciju koja omogućava rad u više-dimenzionalnom prostoru bez da ikad izračuna koordinate u tom višedimenzionalnom prostoru, već koristi unutarnje produkte slika svih parova podataka u prostoru značajki
        <ul>
            <li><i>linear</i> - koristi linearnu plohu da podijeli podatke, formula: $K(X,Y) = X^TY$</li>
            <li><i>poly</i> - koristi polinom da podijeli podatke, formula: $K(X,Y) = (\gamma \cdot X^TY + r)^d$</li>
            <li><i>sigmoid</i> - koristi sigmoidalnu funkciju da podijeli podatke, formula: $K(X,Y) = \tanh(\gamma \cdot X^TY + r)$</li>
            <li><i>rbf</i> -  koristi krivulje koje okružuju podatke i sumira ih tako da granica podjele bude definirana topološkim uvjetom, formula: $K(X,Y) = \exp \bigg( \frac{\mid \mid X - Y\mid \mid^2}{2\sigma^2} \bigg)$</li>
        </ul>
    <li><em>stupanj</em> (u formuli za <i>poly</i> označen s d) - određuje stupanj polinoma</li>
    <li><em>gamma</em> (u formulama za <i>poly</i> i <i>sigmoid</i> označen s $\gamma$) - određuje koeficijent</li>
</ul>

In [17]:
svc_c_params = [0.001, 0.01, 0.1, 1, 10, 1]
kernel_params = ['linear', 'poly', 'sigmoid', 'rbf']
degree_params = [1, 2, 3, 4, 5]
gamma_params = ['scale', 'auto', 0.001, 0.01, 0.1, 1, 10]
svc_params = {'C': svc_c_params, 'kernel': kernel_params, 'degree': degree_params, 'gamma': gamma_params }

#### Boosting

Za boosting ćemo koristiti 4 različita algoritma te usporediti njihove rezultate. Algoritmi koje koristimo su:
<ol>
    <li>XGB (eXtreme Gradient Boosting)</li>
    <li>LGBM (Light Gradient Boosting Method)</li>
    <li>CatBoost (CATegorical Boosting)</li>
    <li>AdaBoost (ADAptive Boosting)</li>
</ol>

##### XGB

Parametri koje podešavamo kod XGB-a:
<ul>
    <li><em>gamma</em> - regularizacijski parametar 'kroz sva stabla'</li>
    <li><em>maksimalna dubina</em> - maksimalna dubina svakog stabla</li>
    <li><em>udio stupaca po stablu</em> - udio značajki koje upotrebljavamo za konstrukciju svakog novog stabla</li>
    <li><em>stopa učenja</em> - određuje koliko koraka treba napraviti (što je stopa manja, imamo više koraka, odnosno sporije učenje, ali možda i precizniji model)</li>
    <li><em>broj procjenitelja</em> - broj stabala koji koristimo prilikom treniranja</li>
</ul>

In [19]:
xgb_gamma = [0, 1, 5, 10]
xgb_max_depth = [1, 2, 3, 4, 5, 7]
xgb_colsample_bytree = [0.3, 0.5, 0.8]
xgb_learning_rate = [0.3, 0.1, 0.05, 0.01]
xgb_n_estimators = [10, 20, 50, 100]
xgb_params = {'gamma': xgb_gamma, 'max_depth': xgb_max_depth, 'colsample_bytree': xgb_colsample_bytree, 
              'learning_rate': xgb_learning_rate, 'n_estimators': xgb_n_estimators}

##### LGBM

Parametri koje podešavamo kod LGBM-a:
<ul>
    <li><em>tip boostinga</em> - određuje koji od sljedećih algoritama koristimo za boosting
        <ul>
            <li><i>gdbt (Gradient Boosting Decision Tree)</i> - tradicionalna stabla odluke trenirana u nekom određenom  redoslijedu (u svakom koraku model pokušava naučiti razliku između stvarnog izlaza i težinske sume predikcija prethodne iteracije)</li>
            <li><i>goss (Gradient-based One Sided Sampling)</i> -  metoda koja se zasniva na odbacivanju instanci s malim gradijentom (jer su te već dobro trenirane), te se fokusira samo na one s većim gradijentom</li>
            <li><i>rf (Random Forest)</i> - klasičan pristup korištenjem slučajnih šuma</li>
            <li><i>dart (Dropouts meets multiple Additive Regression Trees)</i> - novija metoda u koju je dropout tehnika korištena najviše kod dubokih neuronskih mreža</li>
        </ul>
    </li>
    <li><em>maksimalna dubina</em> - ima isto značenje kao i kod XGB-a</li>
    <li><em>udio stupaca po stablu</em> - ima isto značenje kao i kod XGB-a</li>
    <li><em>stopa učenja</em> - ima isto značenje kao i kod XGB-a</li>
    <li><em>broj procjenitelja</em> - ima isto značenje kao i kod XGB-a</li>
</ul>

In [22]:
lgbm_boosting_type = ['gbdt', 'goss', 'rf', 'dart']
lgbm_max_depth = xgb_max_depth
lgbm_learning_rate = xgb_learning_rate
lgbm_colsample_bytree = xgb_colsample_bytree
lgbm_n_estimators = xgb_n_estimators
lgbm_params = {'boosting_type': lgbm_boosting_type, 'max_depth': lgbm_max_depth, 'colsample_bytree': lgbm_colsample_bytree, 
               'learning_rate': lgbm_learning_rate, 'n_estimators': lgbm_n_estimators}


##### CatBoost

Parametri koje podešavamo kod CatBoosta:
<ul>
    <li><em>dubina</em> - isto značenje kao i kod XGB-a</li>
    <li><em>stopa učenja</em> - isto značenje kao i kod XGB-a</li>
    <li><em>broj procjenitelja</em> - isto značenje kao i kod XGB-a</li>
</ul>

In [23]:
cat_depth = xgb_max_depth
cat_learning_rate = xgb_learning_rate
cat_n_estimators = xgb_n_estimators
cat_params = {'depth': cat_depth, 'learning_rate': cat_learning_rate, 'n_estimators': cat_n_estimators }

##### AdaBoost

Parametri koje podešavamo kod AdaBoosta:
<ul>
    <li><em>procjenitelj</em> - procjenitelj kojeg AdaBoost koristi (u našem slučaju stablo odluke)
        <ul>
            <li><i>kriterij</i> - ima isto značenje kao i za stablo odluke</li>
            <li><i>splitter</i> - ima isto značenje kao i za stablo odluke</li>
            <li><i>maksimalna dubina</i> - ima isto značenje kao i za stablo odluke</li>
        </ul>
    </li>
    <li><em>stopa učenja</em> - ima isto značenje kao i kod XGB-a</li>
    <li><em>broj procjenitelja</em> - ima isto značenje kao i kod XGB-a</li>
</ul>

In [24]:
ada_criterion = ['gini', 'entropy']
ada_max_depth = xgb_max_depth
ada_splitter = ['best', 'random']
ada_n_estimators = xgb_n_estimators
ada_learning_rate = xgb_learning_rate
ada_params = {'n_estimators': ada_n_estimators, 'learning_rate': ada_learning_rate}

ada_params = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "base_estimator__max_depth": xgb_max_depth,
              "n_estimators": xgb_n_estimators,
              "learning_rate": xgb_learning_rate
             }